# Test whisper

In [16]:
import openai
import os
from dotenv import load_dotenv
import requests
load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')

audio_url_example = "https://s3.us-west-1.wasabisys.com/blueberryai-input/output_0.mp3"

def convert_speech_to_text_whisper(recording_url):
    # Download the audio file from the URL
    response = requests.get(recording_url)
    audio_file = response.content

    # Save the audio data to a file
    with open("temp.wav", "wb") as file:
        file.write(audio_file)

    # Transcribe the audio using Whisper API
    with open("temp.wav", "rb") as file:
        transcript = openai.Audio.transcribe("whisper-1", file)
    
    return transcript.text

convert_speech_to_text_whisper(audio_url_example)

"Hey, this is People's Gas Company. How can I help you?"

In [ ]:
import json
def string_to_json(string):
    return json.loads(string)
    
def run_hume_transcription(recording_url):
    url = "https://api.hume.ai/v0/batch/jobs"

    payload = "{\"models\":{\"face\":{\"fps_pred\":3,\"prob_threshold\":0.99,\"identify_faces\":false,\"min_face_size\":60,\"save_faces\":false},\"prosody\":{\"granularity\":\"utterance\",\"identify_speakers\":false,\"window\":{\"length\":4,\"step\":1}},\"language\":{\"granularity\":\"word\",\"identify_speakers\":false},\"ner\":{\"identify_speakers\":false}},\"transcription\":{\"language\":null}," + "\"urls\":[\"" + recording_url + "\"],\"notify\":false}"
    headers = {
        "accept": "application/json; charset=utf-8",
        "content-type": "application/json; charset=utf-8",
        "X-Hume-Api-Key": os.getenv("HUME_API_KEY"),
    }
    response = requests.post(url, data=payload, headers=headers)
    job_id = string_to_json(response.text)['job_id']
    
    return job_id

def get_hume_result(job_id):
    while True:
        url = f"https://api.hume.ai/v0/batch/jobs/{job_id}/predictions"

        headers = {
            "accept": "application/json; charset=utf-8",
            "X-Hume-Api-Key": os.getenv("HUME_API_KEY"),
        }

        response = requests.get(url, headers=headers)
        print(string_to_json(response.text))

job_id = run_hume_transcription(audio_url_example)


In [1]:
import requests
import time
import openai
import os
from dotenv import load_dotenv

# Load the variables from the .env file
load_dotenv()

# Set up Twilio client
account_sid = os.getenv('ACCOUNT_SID')
auth_token = os.getenv('AUTH_TOKEN')
twilio_phone_number = os.getenv("TWILIO_PHONE_NUMBER")
recipient_phone_number = os.getenv('RECIPIENT_PHONE_NUMBER')
hume_api_key = os.getenv('HUME_API_KEY')
fliki_api_key = os.getenv('FLIKI_API_KEY')

In [9]:
hume_api_key

'BRlltt9gz2pChEvFQLC38mvEe8jGOoAHuZd4lKeY9vZqydZH'

In [15]:
url = "https://api.fliki.ai/v1/languages"
headers = {
    "Authorization": f"Bearer {fliki_api_key}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)
response.json()

{'success': True,
 'data': [{'_id': '61b8b2f24268666c126babb1', 'name': 'Afrikaans'},
  {'_id': '62b2110fb751a736405225ce', 'name': 'Albanian'},
  {'_id': '61b8b2f24268666c126babb3', 'name': 'Amharic'},
  {'_id': '61b8b2f24268666c126babb5', 'name': 'Arabic'},
  {'_id': '635bc419d0900a036c2e439d', 'name': 'Armenian'},
  {'_id': '62b21780d63b5531a40753f4', 'name': 'Azerbaijani'},
  {'_id': '61b8b2f24268666c126babb7', 'name': 'Bangla/Bengali'},
  {'_id': '635bc666d0900a036c2e6ed2', 'name': 'Basque'},
  {'_id': '62b2188ed63b5531a40757e3', 'name': 'Bosnian'},
  {'_id': '61b8b2f34268666c126babb9', 'name': 'Bulgarian'},
  {'_id': '61b8b2f34268666c126babbb', 'name': 'Burmese'},
  {'_id': '61b8b2f34268666c126babbd', 'name': 'Catalan'},
  {'_id': '61b8b2f44268666c126babbf', 'name': 'Chinese'},
  {'_id': '61b8b2f44268666c126babc1', 'name': 'Croatian'},
  {'_id': '61b8b2f44268666c126babc3', 'name': 'Czech'},
  {'_id': '61b8b2f44268666c126babc5', 'name': 'Danish'},
  {'_id': '61b8b2f54268666c126bab

# Get all the voices

In [ ]:
url = "https://api.fliki.ai/v1/voices"
english_language_id = "61b8b2f54268666c126babc9"
us_dialect_id = "61b8b31c4268666c126bace7"

headers = {
    "Authorization": f"Bearer {fliki_api_key}",
    "Content-Type": "application/json"
}
data = {
    "languageId": english_language_id,
    "dialectId": us_dialect_id
}

response = requests.post(url, headers=headers, json=data)
response.json()

# Conversational Call

In [27]:
conversational_style_id = "6434632c9f50eacb088edafd"
marcus_speaker_id = "643463179f50eacb088edaec"

import requests

content_text = "Um.. yes I'd like to cancel my subscription."

url = "https://api.fliki.ai/v1/generate/text-to-speech"
headers = {
    "Authorization": f"Bearer {fliki_api_key}",
    "Content-Type": "application/json"
}
data = {
    "content": content_text,
    "voiceId": marcus_speaker_id,
    "voiceStyleId": conversational_style_id
}

response = requests.post(url, headers=headers, json=data)

import json

response_text = b'{"success":true,"data":{"audio":"https://storage.googleapis.com/fliki/media/api/648e29e780c5a1088ca021f4/648e335155f0dd4c6d681357.mp3","duration":1.968}}'

# Check the response status code
if response.status_code == 200:
    # Process the response
    audio_data = response.content
    # Do something with the audio data
    response_dict = json.loads(audio_data)

    # Now you can access the dictionary elements
    success = response_dict["success"]
    audio_url = response_dict["data"]["audio"]
    duration = response_dict["data"]["duration"]
    
    print(success, audio_url, duration)
else:
    # Handle the error
    print(f"Request failed with status code {response.status_code}: {response.text}")


True https://storage.googleapis.com/fliki/media/api/648e29e780c5a1088ca021f4/648e33a755f0dd4c6d688d55.mp3 2.016
